In [ ]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import plotly.express as px #для визуализации данных 


In [ ]:
data = pd.read_excel('/Users/eduardgavrilov/Downloads/Тестовое задание.xlsx')
data.drop(columns='good (1)', inplace=True)
# испортируем excel файл в датафрейм и удаляем колонку 


In [ ]:
data.info()
# смотри какие столбцы у нас есть
# тип данных столбцов и заодно проверяем есть ли у нас пустые строки

In [ ]:
data[data.isnull().any(axis=1)]
# проверяем на наличие пустых строки

In [ ]:
data.dropna(inplace=True)
# удаляем полность пустые строки

Далее нужно провести разведывательный анализ

1. Определение пропущенных значений: проверка наличия пропущенных значений в данных и их обработка. 

2. Проверка на наличие дубликатов: проверка данных на дубликаты и их удаление, если они имеются.

Так как у нас у большинства объектов датафрейма тип данных object стандарный метод describe нам ничего особенного не покажет, учитывая что метод describe рассчитывает статистические показатели для числовых признаков. В нашем случает это колонки 'x, y, count, cluster' из всех перечисленных только для колонки count может быть полезет данный метод. 

In [ ]:
data['area'].unique()
data['cluster'].unique()
data['cluster_name'].unique()
data['count'].unique() # в уникальных значениях присутствует 'N\\A' и '-' что 
# является пропусками, нужно данные значения дозаполнить, заполнять будем среднем по area в каждом кластере
data['keyword'].unique()
data['y'].unique()

In [ ]:
data[data['count'] == 'N\\A'].index

In [ ]:
data[data['count'] == '-'].index

Если внимательно посмотреть на признак 'y' с его уникальными значениями, можно обратить внимание что среди чисел есть вот такое значение 0x414fe002 вероятней всего это какая-то ошибка и из-за того что это координата, данную строку придется удалить

In [ ]:
data['count'][49] = round(data[(data['area'] == 'protein') & (data['cluster'] == 0.0)]['count'][0:3].mean())
# заполняем средним значением count в одном area и cluster

In [ ]:
data['count'][217] = round(data[(data['area'] == 'greetings') & (data['cluster'] == 1.0)]['count'][1:3].mean())
# заполняем средним значением count в одном area и cluster

In [ ]:
data = data.drop(99)
# удаляем по индеку в признаке "у" пропущенное значение

Далее приводим колонки к правильному типу данных

In [ ]:
data['cluster'] = data['cluster'].astype('int')
data['count'] = data['count'].astype('int')
data['y'] = data['y'].astype('float')
data['cluster']= data['cluster'].astype('int')

In [ ]:
data.pivot_table(values='cluster', index='area', columns='cluster_name', aggfunc='count')
# данная таблица отражает количество того или иного кластера в каждом area. Это таблица
# помогает с ориентироваться для по количеству требуемых цветов

In [ ]:
colors_hex = [
    "#FF0000", "#FF8C00", "#FFD700", "#00FF00",
    "#4169E1", "#9932CC", "#B22222", "#8B008B",
    "#8B4513", "#808000", "#1E90FF", "#9ACD32",
    "#FF6347", "#00BFFF", "#FF69B4", "#FF4500",
    "#C71585", "#00FF7F", "#DC143C", "#228B22",
    "#00CED1", "#0000CD", "#800080", "#2E8B57",
    "#8B0000", "#008080", "#8A2BE2", "#ADFF2F",
    "#FF1493", "#A0522D", "#6495ED", "#FA8072",
    "#4B0082", "#6B8E23", "#7CFC00", "#FFA500",
    "#483D8B", "#6A5ACD", "#CD853F", "#40E0D0",
    "#9370DB", "#FFDAB9", "#FFA07A", "#20B2AA",
    "#778899", "#00FA9A", "#90EE90", "#D2B48C",
    "#FFC0CB", "#FFFACD", "#FFE4E1", "#FAEBD7",
    "#D8BFD8", "#EE82EE", "#F0E68C", "#CD5C5C",
    "#F08080", "#E6E6FA", "#F5DEB3", "#00FFFF"
]

# создаем список с кодами цветов


colors_dict = {'eligibility': colors_hex[:4],
               'capability': colors_hex[4:8],
               'available': colors_hex[8:12],
               'protein': colors_hex[12:16],
               'winner': colors_hex[16:20],
               'locator': colors_hex[20:24],
               'worlds': colors_hex[24:28],
               'ar\\vr': colors_hex[28:32],
               'twisted': colors_hex[32:36],
               'lithuania': colors_hex[36:40],
               'personnel': colors_hex[40:44],
               'housewives': colors_hex[44:48],
               'dialog': colors_hex[48:52],
               'except': colors_hex[52:56],
               'greetings': colors_hex[56:60]
               }

# для каждого area выделяем из списка по 4 цвета, именно такое количество кластеров в каждой области

Ниже пройдемся циклом for по парам ключ-значение в словаре color_dict. Затем снова циклом for пройдемся по пронумерованному спису area_colors в каждой паре. Далее находим все строки датафрейма где столбец area соотвествует текущей переменной area и аналогично со столбцом cluster и переменной cluster. Далее вводим новый столбец color и добавляе соответствующий код цвета по предварительному фильтру. Таким образом мы по отсортированным строкам сформировали новую нужную нам для построения графиков колонку color

In [ ]:
for area, area_colors in colors_dict.items():
    # print(area, area_colors)
    for cluster, color in enumerate(area_colors):
        # print(cluster)
        data.loc[(data['area'] == area) & (data['cluster'] == cluster), 'color'] = color

Удаляем дубликаты по столбцу keyword для каждого area (без учета кластера!)

In [ ]:
data = data.drop_duplicates(subset=['area', 'keyword'])


In [ ]:
data.to_excel('my_table.xlsx', index=False)
# перезаписываем наш датафрейм так же в excel файл

Далее строим графики для каждой области.

In [683]:
fig_eligibility = px.scatter(data_frame=data[data['area'] == 'eligibility'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "eligibility"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[:4],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_eligibility.show()
fig_eligibility.write_html("fig_eligibility.htmp", full_html=False)

In [679]:
fig_capability = px.scatter(data_frame=data[data['area'] == 'capability'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "capability"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[4:9],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_capability.show("/Users/eduardgavrilov/Documents/IDE/plotly/resume/")

ValueError: 
Invalid named renderer(s) received: ['/Users/eduardgavrilov/Documents/IDE/plotly/resume/']

In [ ]:
fig_available = px.scatter(data_frame=data[data['area'] == 'available'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "available"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[9:13],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_available.show()

In [ ]:
fig_protein = px.scatter(data_frame=data[data['area'] == 'protein'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "protein"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[9:13],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_protein.show()

In [ ]:
fig_protein = px.scatter(data_frame=data[data['area'] == 'protein'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "protein"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[13:17],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_protein.show()

In [ ]:
fig_winner = px.scatter(data_frame=data[data['area'] == 'winner'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "winner"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[17:21],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_winner.show()

In [ ]:
fig_locator = px.scatter(data_frame=data[data['area'] == 'locator'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "locator"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[21:25],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_locator.show()

In [ ]:
fig_worlds = px.scatter(data_frame=data[data['area'] == 'worlds'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "worlds"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[25:29],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_worlds.show()

In [ ]:
fig_ar_vr = px.scatter(data_frame=data[data['area'] == 'ar\\vr'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "ar\\vr"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[29:33],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_ar_vr.show()

In [ ]:
fig_twisted = px.scatter(data_frame=data[data['area'] == 'twisted'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "twisted"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[33:37],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_twisted.show()

In [ ]:
fig_lithuania = px.scatter(data_frame=data[data['area'] == 'lithuania'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "lithuania"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[37:41],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_lithuania.show()

In [ ]:
fig_personnel = px.scatter(data_frame=data[data['area'] == 'personnel'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "personnel"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[41:45],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_personnel.show()

In [ ]:
fig_housewives = px.scatter(data_frame=data[data['area'] == 'housewives'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "housewives"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[45:49],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_housewives.show()

In [ ]:
fig_dialog = px.scatter(data_frame=data[data['area'] == 'dialog'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "dialog"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[49:53],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_dialog.show()

In [ ]:
fig_except = px.scatter(data_frame=data[data['area'] == 'except'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "except"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[53:57],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_except.show()

In [ ]:
fig_greetings = px.scatter(data_frame=data[data['area'] == 'greetings'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "greetings"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[57:],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_greetings.show()

In [ ]:
fig_eligibility = px.scatter(data_frame=data[data['area'] == 'eligibility'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "eligibility"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[:4],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_eligibility.show()

In [ ]:
fig_eligibility = px.scatter(data_frame=data[data['area'] == 'eligibility'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "eligibility"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[:4],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_eligibility.show()

In [ ]:
fig_eligibility = px.scatter(data_frame=data[data['area'] == 'eligibility'], 
                             x='x', 
                             y='y', 
                             title='График рассеивания для области "eligibility"',
                             color='cluster_name',
                             color_discrete_sequence=colors_hex[:4],
                             size='count',
                             width=1500,
                             height=1500,
                             text='keyword',
                             labels={'cluster_name':'Кластеры'}
                             )
fig_eligibility.show()